# WARNING: THIS WILL OVERWRITE THE EXISTING BIRDS.DB FILE
The birds.db specified by database_path will be overwritten. The default value below is the live database for this project.

In [1]:
# Imports a BirdNetPi database into the local birds.db file

import sqlite3
import pandas as pd

pc_username = "<user>"
# Define the path to the database file to be imported (BirdDB.txt)
database_birdnetpi = f"C:/Users/{pc_username}/Downloads/BirdDB.txt"
# Define the path to the database to be written to (birds.db)
database_path = f"C:/Users/{pc_username}/source/repos/BirdNET/birdnet/BirdNET_UI/birds.db"

In [2]:
# [optional] delete all existing records (only use if desired) 

# Connect to the database
conn = sqlite3.connect(database_path)
cursor = conn.cursor()

try:
    # Execute the DELETE statement
    cursor.execute("DELETE FROM birds")
    
    # Commit the changes
    conn.commit()
    print("All records deleted from the birds table.")
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # Close the connection
    cursor.close()
    conn.close()

All records deleted from the birds table.


In [3]:
# Connect to the database
db = sqlite3.connect(database_path)
cursor = db.cursor()
# Create a pandas dataframe from the 'birds' table
birds_df = pd.read_sql_query("SELECT * FROM birds", db)
db.close()
birds_df.head()

,id,scientific_name,common_name,confidence,sighting_time,location_name,latitude,longitude


In [4]:
# Create a pandas dataframe from the BirdNetPi database
birds_df_birdnetpi = pd.read_csv(database_birdnetpi, sep=";")
birds_df_birdnetpi.head()

,Date,Time,Sci_Name,Com_Name,Confidence,Lat,Lon,Cutoff,Week,Sens,Overlap
0,2024-09-23,22:22:24,Helopsaltes ochotensis,Middendorff's Grasshopper Warbler,0.923046,45.0800,-93.0227,0.7,39,1.25,0.0
1,2024-09-23,22:22:39,Helopsaltes ochotensis,Middendorff's Grasshopper Warbler,0.801097,45.0800,-93.0227,0.7,39,1.25,0.0
2,2024-09-23,22:22:39,Helopsaltes ochotensis,Middendorff's Grasshopper Warbler,0.908401,45.0800,-93.0227,0.7,39,1.25,0.0
3,2024-09-23,22:22:39,Helopsaltes ochotensis,Middendorff's Grasshopper Warbler,0.777866,45.0800,-93.0227,0.7,39,1.25,0.0
4,2024-09-25,00:47:57,Corvus brachyrhynchos,American Crow,0.143205,45.0754,-92.9007,0.1,39,1.25,0.0


In [5]:
# Reorganize birds_df_birdnetpi to match the birds.db schema
birds_df_birdnetpi_reordered = birds_df_birdnetpi
birds_df_birdnetpi_reordered['id'] = birds_df_birdnetpi_reordered.index
birds_df_birdnetpi_reordered['scientific_name'] = birds_df_birdnetpi['Sci_Name']
birds_df_birdnetpi_reordered['common_name'] = birds_df_birdnetpi['Com_Name']
birds_df_birdnetpi_reordered['confidence'] = birds_df_birdnetpi['Confidence']
birds_df_birdnetpi_reordered['sighting_time'] = birds_df_birdnetpi['Date'] + ' ' + birds_df_birdnetpi['Time']
birds_df_birdnetpi_reordered['location_name'] = 'BirdNETPi'
birds_df_birdnetpi_reordered['latitude'] = birds_df_birdnetpi['Lat']
birds_df_birdnetpi_reordered['longitude'] = birds_df_birdnetpi['Lon']
# drop the original columns
birds_df_birdnetpi_reordered = birds_df_birdnetpi_reordered.drop(columns=['Sci_Name', 'Com_Name', 'Confidence', 'Date', 'Time', 'Lat', 'Lon', 'Cutoff', 'Week', 'Sens', 'Overlap'])
birds_df_birdnetpi_reordered.head()


,id,scientific_name,common_name,confidence,sighting_time,location_name,latitude,longitude
0,0,Helopsaltes ochotensis,Middendorff's Grasshopper Warbler,0.923046,2024-09-23 22:22:24,BirdNETPi,45.0800,-93.0227
1,1,Helopsaltes ochotensis,Middendorff's Grasshopper Warbler,0.801097,2024-09-23 22:22:39,BirdNETPi,45.0800,-93.0227
2,2,Helopsaltes ochotensis,Middendorff's Grasshopper Warbler,0.908401,2024-09-23 22:22:39,BirdNETPi,45.0800,-93.0227
3,3,Helopsaltes ochotensis,Middendorff's Grasshopper Warbler,0.777866,2024-09-23 22:22:39,BirdNETPi,45.0800,-93.0227
4,4,Corvus brachyrhynchos,American Crow,0.143205,2024-09-25 00:47:57,BirdNETPi,45.0754,-92.9007


In [6]:
# Import the birds_df_birdnetpi_reordered dataframe into the birds.db database
# WARNING: this will overwrite the table schema as well, use the cell below to add values while keeping the schema

# db = sqlite3.connect(database_path)
# birds_df_birdnetpi_reordered.to_sql('birds', db, if_exists='replace', index=False)
# db.close()

In [7]:
conn = sqlite3.connect(database_path)
cursor = conn.cursor()

insert_query = '''
INSERT INTO birds (id, scientific_name, common_name, confidence, sighting_time, location_name, latitude, longitude)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)
'''

data_to_insert = birds_df_birdnetpi_reordered.values.tolist()
cursor.executemany(insert_query, data_to_insert)

conn.commit()
conn.close()